Resources:
- https://quantum-computing.ibm.com/composer/files/new
- https://wybiral.github.io/quantum/
- https://jarrodmcclean.com/basic-quantum-circuit-simulation-in-python/
- https://quantumcomputing.stackexchange.com/questions/29454/why-do-quantum-computing-simulators-have-the-measurement-function

In [82]:
import numpy as np
import scipy as sp

In [83]:
# operator matrices

# identity
Id = np.array([
    [1,0],
    [0,1]
])
# pauli x
X = np.array([
    [0,1],
    [1,0]
])
# pauli y
Y = np.array([
    [0,0-1j],
    [0+1j,0]
])
# pauli z
Z = np.array([
    [1,0],
    [0,-1]
])
# hadamard
H = 1/np.sqrt(2) * np.array([
    [1,1],
    [1,-1]
])
# phase
S = np.array([
    [1,0],
    [0,0+1j]
])
# pi over eight
T = np.array([
    [1,0],
    [0,np.exp((0+1j)*np.pi/4)]
])
# |0><0>
P_0 = np.array([
    [1,0],
    [0,0]
])
# |1><1>
P_1 = np.array([
    [0,0],
    [0,1]
])

In [84]:
class QState:
    def __init__(self, qbit_cnt, info=False):
        self.qbit_cnt = qbit_cnt
        # create state |00...0> with qbit_cnt amount of zeros
        self.state = np.zeros(2**self.qbit_cnt, dtype=complex)
        self.state[0] = 1

        if info:
            max_bin_width = len(bin(len(self.state) - 1)[2:])
            binary_str = bin(0)[2:].zfill(max_bin_width)
            print("Initial state: |{}>".format(binary_str))
    
    # show all states including the impossible one (unless
    # otherwise noted) and show according probabilities
    # DISCLAIMER: rounding/floating point errors may occur
    def show_state_and_probs(self, reduced=False):
        max_len = 0
        for element in self.state:
            if reduced and element == 0+0j:
                continue

            curr_len = max(len(str(element.real)), len(str(element.imag)))
            if curr_len > max_len:
                max_len = curr_len
        
        max_bin_width = len(bin(len(self.state) - 1)[2:])

        for i, element in enumerate(self.state):
            if reduced and element == 0+0j:
                continue

            real_part_str = "{:.{width}f}".format(element.real, width=max_len)
            imaginary_part_str = "{:.{width}f}".format(element.imag, width=max_len)
            
            formatted_element = ("{}+{}i".format(real_part_str, imaginary_part_str) if element.imag >= 0 else
                                 "{}{}i".format(real_part_str, imaginary_part_str))

            binary_str = bin(i)[2:].zfill(max_bin_width)

            prob_str = str(round((np.absolute(element)**2)*100, 4))

            if i > 0 and element.real >= 0:
                print("+{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i > 0 and element.real < 0:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i <= 0 and element.real >= 0:    
                print(" {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            else:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))

    ###
    ### GATE CREATION HELPER ###
    ###
    # e.g. turns [A, B, C, D] into A ⊗ B ⊗ C ⊗ D
    def op_mats_arr_to_tens(self, op_mats_arr):
        result = np.array([[1.0]])
        for op_mat in op_mats_arr:
            result = np.kron(result, op_mat)
        
        return result

    def sing_qbit_op(self, op_mat, pos):
        op_mats_arr = (
              (pos)*[Id]
            + (1)*[op_mat]
            + (self.qbit_cnt-pos-1)*[Id]
        )
        gate = self.op_mats_arr_to_tens(op_mats_arr)
        self.state = np.matmul(gate, self.state)
    
    def ctrl_qbit_op(self, ctrl_pos, targ_pos, targ_op_mat):
        # one of the cases will be zero, and the other one will
        # then represent the final gate

        # case 1
        op_mats_arr1 = (
              (ctrl_pos)*[Id]
            + (1)*[P_0]
            + (self.qbit_cnt-ctrl_pos-1)*[Id]
        )
        gate1 = self.op_mats_arr_to_tens(op_mats_arr1)
        
        # case 2 (split into two possible: first ctrl before | second ctrl after)
        op_mats_arr2 = ((
                  (ctrl_pos)*[Id]
                + (1)*[P_1]
                + (targ_pos-ctrl_pos-1)*[Id]
                + (1)*[targ_op_mat]
                + (self.qbit_cnt-targ_pos-1)*[Id]
            ) if ctrl_pos < targ_pos else (
                  (targ_pos)*[Id]
                + (1)*[targ_op_mat]
                + (ctrl_pos-targ_pos-1)*[Id]
                + (1)*[P_1]
                + (self.qbit_cnt-ctrl_pos-1)*[Id]
        ))
        gate2 = self.op_mats_arr_to_tens(op_mats_arr2)

        gate = gate1 + gate2
        self.state = np.matmul(gate, self.state)

    ###
    ### SINGLE-QUBIT GATES ###
    ###

    def pauli_x(self, pos):
        self.sing_qbit_op(X, pos)

    def pauli_y(self, pos):
        self.sing_qbit_op(Y, pos)

    def pauli_z(self, pos):
        self.sing_qbit_op(Z, pos)

    def hadamard(self, pos):
        self.sing_qbit_op(H, pos)
    
    def phase(self, pos):
        self.sing_qbit_op(S, pos)

    def pi_ov_8(self, pos):
        self.sing_qbit_op(T, pos)

    ###
    ### MULTI-QUBIT GATES ###
    ###
    
    # doesn't matter whether ctrl_pos smaller or bigger than targ_pos
    def cnot(self, ctrl_pos, targ_pos):
        self.ctrl_qbit_op(ctrl_pos, targ_pos, X)

    def cz(self, ctrl_pos, targ_pos):
        self.ctrl_qbit_op(ctrl_pos, targ_pos, Z)
    
    def cp(self, ctrl_pos, targ_pos):
        self.ctrl_qbit_op(ctrl_pos, targ_pos, S)
    
    # quick and dirty test
    def swap(self, pos1, pos2):
        # pos1 > pos2
        # decomposition
        part1 = self.op_mats_arr_to_tens((self.qbit_cnt)*[Id])
        part2 = self.op_mats_arr_to_tens((pos1)*[Id] + (1)*[X] + (pos2-pos1-1)*[Id] + (1)*[X] + (self.qbit_cnt-pos2-1)*[Id])
        part3 = self.op_mats_arr_to_tens((pos1)*[Id] + (1)*[Y] + (pos2-pos1-1)*[Id] + (1)*[Y] + (self.qbit_cnt-pos2-1)*[Id])
        part4 = self.op_mats_arr_to_tens((pos1)*[Id] + (1)*[Z] + (pos2-pos1-1)*[Id] + (1)*[Z] + (self.qbit_cnt-pos2-1)*[Id])
        
        gate = 0.5*(part1 + part2 + part3 + part4)
        self.state = np.matmul(gate, self.state)
        



In [86]:
### Testing

qs = QState(3, True)

qs.pauli_x(0)
qs.show_state_and_probs(True)
print("meow")
qs.swap(0,2)

qs.show_state_and_probs(True)

Initial state: |000>
+1.000+0.000i|100>	-> 100.0%
meow
+1.000+0.000i|010>	-> 100.0%
